In [3]:
import cv2
import numpy as np
import pywt

def wavelet_decompose(img, wavelet='db1', level=2):
    coeffs = pywt.wavedec2(img, wavelet=wavelet, level=level)
    return coeffs

def wavelet_reconstruct(coeffs, wavelet='db1'):
    return pywt.waverec2(coeffs, wavelet=wavelet)

def fuse_coeffs(coeffs1, coeffs2):
    fused = []
    for i in range(len(coeffs1)):
        if i == 0:
            # 저주파 성분 평균
            fused.append((coeffs1[0] + coeffs2[0]) / 2)
        else:
            c1 = coeffs1[i]
            c2 = coeffs2[i]
            fused_subband = []
            for j in range(len(c1)):
                # 고주파 성분은 절댓값이 큰 쪽 선택
                fused_subband.append(np.where(np.abs(c1[j]) > np.abs(c2[j]), c1[j], c2[j]))
            fused.append(tuple(fused_subband))
    return fused

def multi_focus_fusion(img1, img2, wavelet='db1', level=2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    coeffs1 = wavelet_decompose(gray1, wavelet, level)
    coeffs2 = wavelet_decompose(gray2, wavelet, level)

    fused_coeffs = fuse_coeffs(coeffs1, coeffs2)
    fused_img = wavelet_reconstruct(fused_coeffs, wavelet)
    fused_img = np.clip(fused_img, 0, 255).astype(np.uint8)

    return fused_img


In [ ]:

# 이미지 로딩 (같은 크기의 이미지 2장)
img1 = cv2.imread('focus_near.png')
img2 = cv2.imread('focus_far.png')

# 이미지 리사이즈 (같은 크기로 맞춤)
img1 = cv2.resize(img1, (512, 512))
img2 = cv2.resize(img2, (512, 512))

fused = multi_focus_fusion(img1, img2, wavelet='db1', level=3)

cv2.imwrite('fused_result.png', fused)
cv2.imshow('Fused Image', fused)
cv2.waitKey(0)
cv2.destroyAllWindows()
